<a href="https://colab.research.google.com/github/lsteffenel/CHPS0906/blob/main/TP1/4-Entrainer%20un%20mod%C3%A8le%20avec%20PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entraînement des réseaux de neurones

Le réseau que nous avons construit dans la partie précédente n'est pas si intelligent, il ne sait rien de nos chiffres manuscrits. Les réseaux de neurones avec des activations non linéaires fonctionnent comme des approximateurs de fonctions universelles, où une fonction mappe votre entrée à la sortie. Par exemple, une fonction qui relie des images de chiffres manuscrits aux probabilités de classe. La puissance des réseaux de neurones est que nous pouvons les entraîner à approximer cette fonction... À vrai dire, on pourrait approximer n'importe quelle fonction à condition de disposer de suffisamment de données et de temps de calcul.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/function_approx.png?raw=1" width=500px>

Au début, le réseau est très naïf, il ne connaît pas la fonction qui mappe les entrées aux sorties. Nous entraînons le réseau en lui montrant des exemples de données réelles, puis en ajustant les paramètres du réseau de manière à ce qu'il se rapproche de cette fonction.

Pour trouver ces paramètres, nous devons savoir dans quelle mesure le réseau prédit mal les sorties réelles. Pour cela, nous calculons une **fonction de perte** (également appelée coût ou **loss**), une mesure de notre erreur de prédiction. Par exemple, la perte quadratique moyenne (RMSE) est souvent utilisée dans les problèmes de régression et de classification binaire

$$
\large \ell = \frac{1}{2n}\sum_i^n{\left(y_i - \hat{y}_i\right)^2}
$$

où $n$ est le nombre d'exemples d'apprentissage, $y_i$ sont les vraies étiquettes et $\hat{y}_i$ sont les étiquettes prédites.

En minimisant cette perte par rapport aux paramètres du réseau, nous pouvons trouver des configurations où la perte est minimale et le réseau est capable de prédire les étiquettes correctes avec une grande précision. Nous trouvons ce minimum en utilisant un processus appelé **descente de gradient**. Le gradient est la pente de la fonction de perte et pointe dans la direction du changement le plus rapide. Pour atteindre le minimum en un minimum de temps, nous voulons ensuite suivre le gradient (vers le bas). Vous pouvez penser à cela comme descendre une montagne en suivant la pente la plus raide jusqu'à la base.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/gradient_descent.png?raw=1' width=350px>

## Backpropagation

Pour les réseaux à couche unique, la descente de gradient est simple à mettre en œuvre. Cependant, elle est plus compliquée pour les réseaux neuronaux multicouches plus profonds comme celui que nous avons construit. Assez compliqué pour qu'il ait fallu environ 30 ans avant que les chercheurs ne découvrent comment entraîner des réseaux multicouches.

L'entraînement des réseaux multicouches se fait par **rétropropagation**, qui n'est en fait qu'une application de la règle de la chaîne du calcul. Il est plus facile de comprendre si nous convertissons un réseau à deux couches en une représentation graphique.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/backprop_diagram.png?raw=1' width=550px>

Dans le passage en avant à travers le réseau, nos données et nos opérations vont ici de bas en haut. Nous passons l'entrée $x$ par une transformation linéaire $L_1$ avec des poids $W_1$ et des biais $b_1$. La sortie passe ensuite par l'opération sigmoïde $S$ et une autre transformation linéaire $L_2$. Enfin, nous calculons la perte $\ell$. Nous utilisons la perte comme mesure de la mauvaise qualité des prédictions du réseau. L'objectif est alors d'ajuster les pondérations et les biais pour minimiser la perte.

Pour entraîner les pondérations avec la descente de gradient, nous propageons le gradient de la perte vers l'arrière à travers le réseau. Chaque opération a un gradient entre les entrées et les sorties. Lorsque nous renvoyons les gradients vers l'arrière, nous multiplions le gradient entrant par le gradient de l'opération. Mathématiquement, il s'agit simplement de calculer le gradient de la perte par rapport aux pondérations à l'aide de la règle de la chaîne.

$$
\large \frac{\partial \ell}{\partial W_1} = \frac{\partial L_1}{\partial W_1} \frac{\partial S}{\partial L_1} \frac{\partial L_2}{\partial S} \frac{\partial \ell}{\partial L_2}
$$

Nous mettons à jour nos poids en utilisant ce gradient avec un taux d'apprentissage $\alpha$ (**learning rate**).

$$
\large W^\prime_1 = W_1 - \alpha \frac{\partial \ell}{\partial W_1}
$$

Le learning rate $\alpha$ est défini de telle sorte que les étapes de mise à jour des poids soient suffisamment petites pour que la méthode itérative s'établisse à un minimum.

## Loss dans PyTorch

Commençons par voir comment nous calculons le *loss* avec PyTorch. Grâce au module `nn`, PyTorch fournit des formules de loss telles que la perte d'entropie croisée (`nn.CrossEntropyLoss`). Vous verrez généralement la perte attribuée à `criterion`.

Comme indiqué dans la dernière partie, avec un problème de classification tel que MNIST, nous utilisons la fonction softmax pour prédire les probabilités de classe. Avec une sortie softmax, vous souhaitez utiliser l'entropie croisée comme fonction de perte. Pour calculer réellement la perte, vous définissez d'abord le critère, puis vous transmettez la sortie de votre réseau et les étiquettes correctes.

Quelque chose de vraiment important à noter ici. En regardant [la documentation de `nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss),

> Ce critère combine `nn.LogSoftmax()` et `nn.NLLLoss()` dans une seule classe.
>
> L'entrée doit contenir des scores pour chaque classe.

Cela signifie que nous devons transmettre la sortie brute de notre réseau pour la fonction de calcul de la perte, et non la sortie de la fonction softmax. Cette sortie brute est généralement appelée *logits* ou *scores*. Nous utilisons les logits car softmax vous donne des probabilités qui seront souvent très proches de zéro ou de un, mais les nombres à virgule flottante ne peuvent pas représenter avec précision des valeurs proches de zéro ou de un ([en savoir plus ici](https://docs.python.org/3/tutorial/floatingpoint.html)). Il est généralement préférable d'éviter de faire des calculs avec des probabilités, nous utilisons généralement des probabilités logarithmiques.

In [ ]:
# The MNIST datasets are hosted on yann.lecun.com that has moved under CloudFlare protection
# Run this script to enable the datasets download
# Reference: https://github.com/pytorch/vision/issues/1938

from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
# Build a feed-forward network
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10))

# Define the loss
criterion = nn.CrossEntropyLoss()

# Get our data
dataiter = iter(trainloader)

images, labels = next(dataiter)

# Flatten images
images = images.view(images.shape[0], -1)

# Forward pass, get our logits
logits = model(images)
# Calculate the loss with the logits and the labels
loss = criterion(logits, labels)

print(loss)

En raison de cette spécificité de la fonction `nn.CrossEntropyLoss`, il est plus pratique de construire le modèle avec une sortie log-softmax en utilisant `nn.LogSoftmax` ou `F.log_softmax` ([documentation](https://pytorch.org/docs/stable/nn.html#torch.nn.LogSoftmax)). Vous pouvez ensuite obtenir les probabilités réelles en prenant l'exponentielle `torch.exp(output)`. Avec une sortie log-softmax, il faut utiliser la perte `nn.NLLLoss` (*negative log likelihood loss*) ([documentation](https://pytorch.org/docs/stable/nn.html#torch.nn.NLLLoss)).

>**Exercice :** Construisez un modèle qui renvoie le log-softmax comme sortie et calculez la perte en utilisant la perte `NLLLoss`. Notez que pour `nn.LogSoftmax` et `F.log_softmax`, vous devrez définir l'argument du mot-clé `dim` de manière appropriée. `dim=0` calcule le softmax sur les lignes, de sorte que chaque colonne totalise 1, tandis que `dim=1` calcule sur les colonnes de sorte que chaque ligne totalise 1. Pensez à ce que vous voulez que soit le résultat et choisissez `dim` de manière appropriée.

In [ ]:
# TODO: Build a feed-forward network
model =

# TODO: Define the loss
criterion =

### Run this to check your work
# Get our data
dataiter = iter(trainloader)

images, labels = next(dataiter)

# Flatten images
images = images.view(images.shape[0], -1)

# Forward pass, get our logits
logits = model(images)
# Calculate the loss with the logits and the labels
loss = criterion(logits, labels)

print(loss)

## Autograd

Maintenant que nous savons comment calculer une perte, comment l'utiliser pour effectuer une rétropropagation ? Torch fournit un module, `autograd`, pour calculer automatiquement les gradients des tenseurs. Nous pouvons l'utiliser pour calculer les gradients de tous nos paramètres par rapport à la perte. Autograd fonctionne en gardant une trace des opérations effectuées sur les tensors, puis en revenant en arrière dans ces opérations, en calculant les gradients au fur et à mesure. Pour vous assurer que PyTorch garde une trace des opérations sur un tensor et calcule les gradients, vous devez définir `requires_grad = True` sur le tensor. Vous pouvez le faire à la création avec le mot-clé `requires_grad`, ou à tout moment avec `x.requires_grad_(True)`.

Vous pouvez également désactiver les gradients pour un bloc de code avec le contenu de `torch.no_grad()` :
```python
x = torch.zeros(1, require_grad=True)
>>> avec torch.no_grad() :
... y = x * 2
>>> y.requires_grad
False
```

Vous pouvez également activer ou désactiver complètement les gradients avec `torch.set_grad_enabled(True|False)`.

Les gradients sont calculés par rapport à une variable `z` avec `z.backward()`. Cela effectue un passage en arrière à travers les opérations qui ont créé `z`.

In [ ]:
x = torch.randn(2,2, requires_grad=True)
print(x)

In [ ]:
y = x**2
print(y)

Ci-dessous, nous pouvons voir l'opération qui a créé « y », une opération de puissance « PowBackward0 ».

In [ ]:
## grad_fn shows the function that generated this variable
print(y.grad_fn)

Le module autograd garde une trace de ces opérations et sait calculer le gradient pour chacune d'elles. De cette façon, il est capable de calculer les gradients d'une chaîne d'opérations, par rapport à un tensor quelconque. Réduisons le tensor `y` à une valeur scalaire, celle de la moyenne.

In [ ]:
z = y.mean()
print(z)

Vous pouvez vérifier les dégradés pour « x » et « y » mais ils sont actuellement vides.

In [ ]:
print(x.grad)

Pour calculer les gradients, vous devez exécuter la méthode `.backward` sur une variable, `z` par exemple. Cela calculera le gradient de `z` par rapport à `x`

$$
\frac{\partial z}{\partial x} = \frac{\partial}{\partial x}\left[\frac{1}{n}\sum_i^n x_i^2\right] = \frac{x}{2}
$$

In [ ]:
z.backward()
print(x.grad)
print(x/2)

These gradients calculations are particularly useful for neural networks. For training we need the gradients of the cost with respect to the weights. With PyTorch, we run data forward through the network to calculate the loss, then, go backwards to calculate the gradients with respect to the loss. Once we have the gradients we can make a gradient descent step.

## Loss et Autograd ensemble

Lorsque nous créons un réseau avec PyTorch, tous les paramètres sont initialisés avec `requires_grad = True`. Cela signifie que lorsque nous calculons la perte et appelons `loss.backward()`, les gradients des paramètres sont calculés. Ces gradients sont utilisés pour mettre à jour les poids avec la descente de gradient. Vous pouvez voir ci-dessous un exemple de calcul des gradients à l'aide d'un passage en arrière.

In [ ]:
# Build a feed-forward network
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.view(images.shape[0], -1)

logits = model(images)
loss = criterion(logits, labels)

In [ ]:
print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model[0].weight.grad)

## Entraînement du réseau !

Il reste un dernier élément : un optimiseur, que nous utiliserons pour mettre à jour les poids avec les gradients. Nous les obtenons à partir du package [`optim`](https://pytorch.org/docs/stable/optim.html) de PyTorch. Par exemple, nous pouvons utiliser la descente de gradient stochastique avec `optim.SGD`. Vous pouvez voir comment définir un optimiseur ci-dessous.

In [ ]:
from torch import optim

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)

Maintenant que nous savons comment utiliser toutes les parties individuelles, il est temps de voir comment elles fonctionnent ensemble. Considérons une seule étape d'apprentissage avant de parcourir toutes les données. Le processus général avec PyTorch :

* Effectuez un passage en avant à travers le réseau
* Utilisez la sortie du réseau pour calculer la perte
* Effectuez un passage en arrière à travers le réseau avec `loss.backward()` pour calculer les gradients
* Faites un pas avec l'optimiseur pour mettre à jour les poids

Ci-dessous, nous allons parcourir une étape d'entraînement et imprimer les poids et les gradients afin que vous puissiez voir comment cela change. Notez que nous avons également rajouté une ligne de code `optimizer.zero_grad()`. Lorsque vous effectuez plusieurs passes en arrière avec les mêmes paramètres, les gradients sont accumulés. Cependant, quand on change les données (un nouveau batch, par exemple), il faut mettre à zéro les gradients sinon on conservera les gradients des batchs d'entraînement précédents.

In [ ]:
print('Initial weights - ', model[0].weight)

dataiter = iter(trainloader)
images, labels = next(dataiter)
images.resize_(64, 784)

# Clear the gradients, do this because gradients are accumulated
optimizer.zero_grad()

# Forward pass, then backward pass, then update weights
output = model(images)
loss = criterion(output, labels)
loss.backward()
print('Gradient -', model[0].weight.grad)

In [ ]:
# Take an update step and view the new weights
optimizer.step()
print('Updated weights - ', model[0].weight)

### Entraînement pour de vrai

Nous allons maintenant mettre cet algorithme dans une boucle afin de pouvoir parcourir toutes les images. Un passage sur l'ensemble des données est appelé une **Epoch**. Nous allons donc ici parcourir `trainloader` pour obtenir nos batches d'entraînement. Pour chaque batch, nous effectuerons un passage d'entraînement au cours duquel nous calculerons la perte, effectuerons un passage en arrière et mettrons à jour les poids.

>**Exercice :** implémentez l'entraînement pour notre réseau. Si vous l'avez implémenté correctement, vous devriez voir la perte d'entraînement diminuer à chaque epoch.

In [ ]:
## Your solution here

model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)

        # TODO: Training pass

        loss =

        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

Avec le réseau entraîné, nous pouvons vérifier ses prédictions.

In [ ]:
%matplotlib inline
import helper

dataiter = iter(trainloader)
images, labels = next(dataiter)

img = images[0].view(1, 784)
# Turn off gradients to speed up this part
with torch.no_grad():
    logps = model(img)

# Output of the network are log-probabilities, need to take exponential for probabilities
ps = torch.exp(logps)
helper.view_classify(img.view(1, 28, 28), ps)

Notre réseau est maintenant brillant. Il peut prédire avec précision les chiffres de nos images.

Avant de nous lancer sur d'autres modèles plus complexes, les prochains sujets de TP vous aideront à visualiser le fonctionnement de certains composants (convolutions, pooling).